# The code in this notebook was used to alter an Sqlite database. It is not a good idea to run all cells as you might another notebook.

In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import re
import sqlite3
import os

In [ ]:
con = sqlite3.connect('C:\sqlite\eclog.db')

In [ ]:
con.enable_load_extension(True)
con.load_extension("regexp")
con.enable_load_extension(False)        

In [ ]:
cur = con.cursor()

In [ ]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())

In [ ]:
cur.execute('''SELECT DISTINCT UserAgent 
            FROM http_raw
            WHERE UserAgent LIKE '%bot%';''')
cur.fetchall()

Unfortunately CUBOT is a brand of cell phone and we do not want to lose this data. Do genuine bots ever add an item to the cart? 

In [ ]:
cur.execute('''CREATE TABLE managementIps AS SELECT DISTINCT IpId
            FROM http_raw
            WHERE Uri LIKE '/zarzadzanie/%';''')
con.commit()

In [ ]:
cur.execute('PRAGMA table_info(managementIps);')
cur.fetchall()

In [ ]:
cur.execute('''CREATE TABLE notManagement AS SELECT *
            FROM http_raw A
            LEFT JOIN managementIps B
            ON A.IpId = B.IpId
            WHERE B.IpId IS NULL;''')
con.commit()

In [ ]:
cur.execute('''SELECT DISTINCT UserAgent
            FROM notManagement
            WHERE Uri LIKE '%do_koszyka.%' AND UserAgent LIKE '%bot%';''')
cur.fetchall()

This is good, it means we do not need to exclude 'bots' if we limit to 'customers'.

In [ ]:
cur.execute('''CREATE TABLE customerIps AS SELECT DISTINCT IpId
            FROM http_raw
            WHERE Uri LIKE '%do_koszyka.%';''')
con.commit()

In [ ]:
cur.execute('''CREATE TABLE customerHTTP AS SELECT *
            FROM http_raw A
            INNER JOIN customerIps B
            ON A.IpId = B.IpId;''')
con.commit()

In [ ]:
ticks = 30*60*(10**7)
ticks

In [ ]:
cur.execute('''SELECT IpId, TimeStamp, COUNT(*)
            FROM customerHTTP
            GROUP BY IpId, TimeStamp
            HAVING COUNT(*) > 1;''')
cur.fetchall()

In [ ]:
cur.execute('''SELECT *
            FROM customerHTTP
            WHERE IpId = '1000PL';''')
cur.fetchall()

In [ ]:
cur.execute('''CREATE TABLE preSessions AS SELECT IpId AS IpId,
            TimeStamp AS TimeStamp,
            Uri AS Uri,
            HttpMethod AS HttpMethod,
            Referrer as Referrer,
            lag(IpId) OVER win as LagIpId,
            lag(TimeStamp) OVER win as LagTimeStamp,
            lead(IpId) OVER win as LeadIpId,
            lead(TimeStamp) OVER win as LeadTimeStamp
            FROM customerHTTP
            WINDOW win AS (ORDER BY IpId, TimeStamp);
            ''')
con.commit()

In [ ]:
con.close()

In [ ]:
cur.execute('''ALTER TABLE preSessions
            ADD COLUMN Event AS (CASE
            WHEN IpId != LagIpId AND IpId != LeadIpId THEN 'Flit'
            WHEN TimeStamp - LagTimeStamp > 18000000000 AND LeadTimeStamp - TimeStamp > 18000000000 THEN 'Flit'
            WHEN IpId != LagIpId AND LeadTimeStamp - TimeStamp > 18000000000 THEN 'Flit'
            WHEN IpId != LeadIpId AND TimeStamp - LagTimeStamp > 18000000000 THEN 'Flit'
            WHEN IpId != LagIpId OR TimeStamp - LagTimeStamp > 18000000000 THEN 'Arrive'
            WHEN IpId != LeadIpId OR LeadTimeStamp - TimeStamp > 18000000000 THEN 'Leave'
            END);''')
con.commit()

In [ ]:
cur.execute('PRAGMA table_xinfo(preSessions);')
cur.fetchall()

In [ ]:
def getProduct(y):
    product = re.search('p-[0-9_]+|r-[0-9_]+|kartapdf-[0-9_]+|c-[0-9_]+|m-[0-9_]+', y)
    if product != None:
        return product.group(0)

In [ ]:
con.create_function('getProd', 1, getProduct, deterministic=True)

In [ ]:
cur.execute('''ALTER TABLE preSessions
            ADD COLUMN UriProduct TEXT AS (getProd(Uri));''')
con.commit()

In [ ]:
cur.execute('''ALTER TABLE preSessions
            ADD COLUMN RefProduct TEXT AS (getProd(Referrer));''')
con.commit()

In [ ]:
cur.execute('''ALTER TABLE preSessions
            ADD COLUMN CurrentProduct TEXT AS (CASE
            WHEN UriProduct IS NOT NULL THEN UriProduct
            WHEN RefProduct IS NOT NULL THEN RefProduct
            END);''')
con.commit()

In [ ]:
cur.execute('''ALTER TABLE preSessions
            ADD COLUMN BeginSession INTEGER AS (CASE
            WHEN Event ='Arrive' OR Event = 'Flit' THEN 1
            ELSE 0
            END);''')
con.commit()

In following line, CREATE TABLE used because there was a problem using the window function for ALTER TABLE with ADD COLUMN. Presumably because it was unclear when the ORDER BY should be applied and to what table. 

In [ ]:
cur.execute('''CREATE TABLE preSessions_ AS SELECT *,
            SUM(BeginSession)
            OVER (ORDER BY IpId, TimeStamp ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW)
            AS SessionId 
            FROM preSessions;
            ''')
con.commit()

In [ ]:
cur.execute('''CREATE TABLE preSessions__ AS SELECT *,
            max(CurrentProduct) OVER (PARTITION BY SessionID, Grp 
                ORDER BY TimeStamp 
                ROWS UNBOUNDED PRECEDING) AS Product_ 
            FROM (SELECT *, max(CASE 
                WHEN CurrentProduct IS NOT NULL THEN TimeStamp
                END) OVER (PARTITION BY SessionId 
                    ORDER BY TimeStamp ROWS UNBOUNDED PRECEDING) AS Grp
                FROM preSessions_);
            ''')
con.commit()

In [ ]:
cur.execute('''ALTER TABLE preSessions__
            ADD COLUMN Action TEXT AS (CASE
            WHEN HttpMethod = 'POST' AND Uri LIKE '%do_koszyka.%' THEN 'add_to_cart'
            WHEN HttpMethod = 'POST' AND Uri LIKE '%koszyk_usun%' THEN 'delete_cart'
            WHEN HttpMethod = 'POST' AND Uri LIKE '%usun_z_koszyka%' THEN 'remove_from_cart'
            WHEN HttpMethod = 'POST' AND Uri LIKE '%zamowienie_realizacja%' THEN 'order'
            WHEN HttpMethod = 'POST' AND Uri LIKE '%do_zmiany_danych_zamowienie%' THEN 'change_order'
            END);''')
con.commit()

In [ ]:
cur.execute('''ALTER TABLE preSessions__
            ADD COLUMN Product TEXT AS (CASE
            WHEN Action='add_to_cart' AND CurrentProduct IS NOT NULL THEN CurrentProduct
            WHEN Action='add_to_cart' THEN Product_
            END);''')
con.commit()

In [ ]:
cur.execute('''CREATE TABLE Sessions AS SELECT SessionID, IpId, TimeStamp, Event, Action, Product
            FROM preSessions__
            WHERE Event IS NOT NULL OR Action IS NOT NULL;
            ''')
con.commit()

In [ ]:
cur.execute('SELECT COUNT(*) FROM Sessions;')
cur.fetchall()

In [ ]:
cur.execute('SELECT * FROM Sessions;')
sessions = pd.DataFrame(cur.fetchall(), columns=['SessionID', 'IpId', 'TimeStamp', 'Event', 'Action', 'Product'])

In [ ]:
sessions.to_csv('sessions.csv')

In [ ]:
con.close()